# Description

In this tutorial, we will learn how to load data MRI images saved in the `.nii` format using `nibabel` package. We will use the sample data that comes with the `topology_radiomics` package. The sample data is supposed to represent a lesion. After, we will compute the surface features using the `topology_radiomics` package. Finally, we will visualize the surface mesh of all the surface measures.

The tutorial will take the following steps:

1. Importing the packages
1. Configuring the logging to view output generated by the bric_radiomics package
1. Importing the `plotly` visualization package
1. Converting `.nii` files to a numpy array
1. Converting numpy array to a sanitized mask using bric_radiomics utility functions
1. Retrieve the surface measures as a pandas dataframe
1. Visualize the mesh of the lesion with the surface measure overlayed on each point on the mesh


In [ ]:
import topology_radiomics as rad
import nibabel as ni
import plotly
import pandas as pd
import logging
FORMAT = '%(asctime)-15s %(levelname)s %(funcName)s  %(message)s'
logging.basicConfig(format=FORMAT, level=logging.DEBUG)

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
init_notebook_mode()

In [ ]:
path = rad.get_sample_nii_path()
ni_image = ni.load(path)
mri_image = ni_image.get_fdata()

In [ ]:
config = rad.MorphologyConfig()

In [ ]:
mask_12 = rad.convert_volume_into_mask(mri_image,merge_labels=[1,2])
lesion = rad.compute_morphology_features(mask_12, config)

In [ ]:
df = lesion.to_DF()
df

In [ ]:
def create_3dmesh(lesion, df, column_name):
    x_coordinates = df["x"]
    y_coordinates = df["y"]
    z_coordinates = df["z"]

    x_face_index = lesion.isosurface.faces[:, 0]
    y_face_index = lesion.isosurface.faces[:, 1]
    z_face_index = lesion.isosurface.faces[:, 2]
    
    vals = df[column_name]
        
    plotly_mesh = go.Mesh3d(
        x=x_coordinates,
        y=y_coordinates,
        z=z_coordinates,
        i=x_face_index,
        j=y_face_index,
        k=z_face_index,
        text=list(zip(x_coordinates,y_coordinates,z_coordinates,vals)),
        intensity=vals
    )
    return plotly_mesh

def all_meshes(lesion, df):
    plotly_3d_meshes = []
    columns = [
           "curvedness",
           "shape_index", 
           "sharpness",
           "total_curvature"
        ]
    for measure_name in columns:
        plotly_3d_mesh = create_3dmesh(lesion, df, measure_name)
        plotly_3d_meshes.append(plotly_3d_mesh)
    return plotly_3d_meshes


def generate_figure(plotly_3d_meshes):
    fig = go.Figure(data=plotly_3d_meshes)
    fig.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                direction="right",
                active=0,
                x=0.57,
                y=1.2,
                buttons=list([
                    dict(label="curvedness",
                         method="update",
                         args=[{"visible": [True, False, False, False]},
                               {"title": "Curvedness"}]),
                    dict(label="shape index",
                         method="update",
                         args=[{"visible": [False, True, False, False]},
                               {"title": "Shape Index"}]),
                    dict(label="sharpness",
                         method="update",
                         args=[{"visible": [False, False, True, False]},
                               {"title": "Sharpness"}]),
                    dict(label="total curvature",
                         method="update",
                         args=[{"visible": [False, False, False, True]},
                               {"title": "Total Curvature"}]),
                ]),
            )
        ]
    )
    return fig

meshes = all_meshes(lesion, df)
fig = generate_figure(meshes)
fig.show()